In [ ]:
import os
import sys
import argparse

import imageio
import torch
from tqdm.notebook import tqdm

sys.path.append("../StyleCLIP_modular")
from style_clip import Imagine, create_text_path

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument("--batch_size", default=32, type=int)
parser.add_argument("--gradient_accumulate_every", default=1, type=int)
parser.add_argument("--save_every", default=1, type=int)
parser.add_argument("--epochs", default=1, type=int)
parser.add_argument("--story_start_words", default=5, type=int)
parser.add_argument("--story_words_per_epoch", default=5, type=int)
parser.add_argument("--style", default="../stylegan2-ada-pytorch/VisionaryArt.pkl", type=str, choices=["faces (ffhq config-f)", "../stylegan2-ada-pytorch/VisionaryArt.pkl"])
parser.add_argument("--lr_schedule", default=0, type=int)
parser.add_argument("--start_image_steps", default=1000, type=int)
parser.add_argument("--iterations", default=100, type=int)
args = vars(parser.parse_args({}))

args["opt_all_layers"] = 1
args["lr_schedule"] = 1
args["noise_opt"] = 0
args["reg_noise"] = 0
args["seed"] = 1

args["model_type"] = "vqgan"
args["iterations"] = 200
args["save_every"] = 1
args["start_img_loss_weight"] = 0.0

args["lr"] = 0.1

#run(img="base_images/aicpa_logo_black.jpg", start_image_path="base_images/stance.jpg", args=args)
#run(img="base_images/aicpa_logo_black.jpg", start_image_path="base_images/earth.jpg", args=args)
#run(img="base_images/earth.jpg", start_image_path="base_images/aicpa_logo_black.jpg", args=args)

In [ ]:
args["start_image_steps"] = 500
args["sideX"] = 320
args["sideY"] = 320

path_dict = {"logo_black": "base_images/aicpa_logo_black.jpg",
        "logo_purple": "base_images/aicpa_logo_purple.jpg",
        "earth": "base_images/earth.jpg",
        "stance": "base_images/stance.jpg",
        }
latent_dict = {}

for key in path_dict:
    print(key)
    args["start_image_path"] = path_dict[key]
    imagine = Imagine(
                save_progress=False,
                open_folder=False,
                save_video=False,
                **args
               )
    img = imagine.prime_image()
    latents = imagine.model.model.latents.detach().cpu()
    latent_dict[key] = latents

In [ ]:
# earth, logo_black, logo_purple, stance

start = "earth"
end = "stance"
steps = 100

start_latent, end_latent = latent_dict[start], latent_dict[end]

# interpolate
# Obtain evenly-spaced ratios between 0 and 1
linspace = torch.linspace(0, 1, steps)

# Generate arrays for interpolation using ratios
latent_transition = [(1 - l) * start_latent + l * end_latent for l in linspace]

In [ ]:
# generate images from latents
images = []
for latent in tqdm(latent_transition):
    model = imagine.model.model
    model.latents = latent.to("cuda")
    image = model(return_loss=False)
    image = image.detach().cpu().squeeze(0).permute(1, 2, 0).clamp(0, 1) * 255
    image = image.type(torch.uint8).clamp(0, 255)
    images.append(image)

In [ ]:
folder = "transitions"
os.makedirs(folder, exist_ok=1)
path = os.path.join(folder, f"{start}_to_{end}_{steps}.mp4")
imageio.mimwrite(path, images)